In [1]:
import pandas as pd
from tool_datetime import next_datetimes

In [2]:
# Define the path where the results arising from this analysis will be saved.
path_to_save_data = "./time-series/"

In [3]:
# Read the dataframe regarding the population.
df = pd.read_csv("./wfp_data/Population/population.csv")
df.head()

,adm0_name,adm0_code,adm1_name,adm1_code,population
0,Yemen,269,Abyan,3407,583193
1,Yemen,269,Al Dhale'e,3408,753302
2,Yemen,269,Aden,3409,953907
3,Yemen,269,Al Bayda,3410,770257
4,Yemen,269,Al Hudaydah,3411,3315493


In [4]:
df.reset_index(inplace = True)
df.rename(columns = {"adm0_name": "Country", "adm1_name": "AdminStrata", "population": "Population", "index": "Datetime"}, inplace = True)
df.drop(columns = ["adm0_code", "adm1_code"], inplace = True)
df.head()

,Datetime,Country,AdminStrata,Population
0,0,Yemen,Abyan,583193
1,1,Yemen,Al Dhale'e,753302
2,2,Yemen,Aden,953907
3,3,Yemen,Al Bayda,770257
4,4,Yemen,Al Hudaydah,3315493


In [5]:
df.AdminStrata.unique()

array(['Abyan', "Al Dhale'e", 'Aden', 'Al Bayda', 'Al Hudaydah',
       'Al Jawf', 'Al Maharah', 'Al Mahwit', 'Amran', 'Dhamar', 'Hajjah',
       'Ibb', 'Lahj', 'Marib', "Sa'ada", "Sana'a", 'Shabwah', 'Taizz',
       'Amanat Al Asimah', 'Raymah', 'Hadramaut', 'Socotra'], dtype=object)

In [6]:
# Create multi-column dataframe.
df = df.set_index(["Datetime", "Country", "AdminStrata"]).unstack(["Country", "AdminStrata"])
df = df.reindex(sorted(df.columns), axis = 1)
df.columns = df.columns.droplevel(0)
df = df.fillna(method = "backfill").dropna()

In [7]:
n_days = 1400

In [8]:
df = pd.concat([df]*(n_days))
df.index = next_datetimes("2017-01-01", n_days, "D")
df.index.name = "Datetime"
freq = "D"
df.index.freq = freq
df.head()

Country         Yemen                                                       \
AdminStrata     Abyan      Aden  Al Bayda Al Dhale'e Al Hudaydah   Al Jawf   
Datetime                                                                     
2017-01-02   583193.0  953907.0  770257.0   753302.0   3315493.0  589218.0   
2017-01-03   583193.0  953907.0  770257.0   753302.0   3315493.0  589218.0   
2017-01-04   583193.0  953907.0  770257.0   753302.0   3315493.0  589218.0   
2017-01-05   583193.0  953907.0  770257.0   753302.0   3315493.0  589218.0   
2017-01-06   583193.0  953907.0  770257.0   753302.0   3315493.0  589218.0   

Country                                                       ...             \
AdminStrata Al Maharah Al Mahwit Amanat Al Asimah      Amran  ...     Hajjah   
Datetime                                                      ...              
2017-01-02    159396.0  749952.0        2963191.0  1173354.0  ...  2187109.0   
2017-01-03    159396.0  749952.0        2963191.0  1173354.0  ...  2187109.0   
2017-01-04    159396.0  749952.0        2963191.0  1173354.0  ...  2187109.0   
2017-01-05    159396.0  749952.0        2963191.0  1173354.0  ...  2187109.0   
2017-01-06    159396.0  749952.0        2963191.0  1173354.0  ...  2187109.0   

Country                                                                     \
AdminStrata        Ibb       Lahj     Marib    Raymah    Sa'ada     Sana'a   
Datetime                                                                     
2017-01-02   3016766.0  1027981.0  368454.0  622101.0  923595.0  1497112.0   
2017-01-03   3016766.0  1027981.0  368454.0  622101.0  923595.0  1497112.0   
2017-01-04   3016766.0  1027981.0  368454.0  622101.0  923595.0  1497112.0   
2017-01-05   3016766.0  1027981.0  368454.0  622101.0  923595.0  1497112.0   
2017-01-06   3016766.0  1027981.0  368454.0  622101.0  923595.0  1497112.0   

Country                                    
AdminStrata   Shabwah  Socotra      Taizz  
Datetime                                   
2017-01-02   649970.0  58723.0  3011994.0  
2017-01-03   649970.0  58723.0  3011994.0  
2017-01-04   649970.0  58723.0  3011994.0  
2017-01-05   649970.0  58723.0  3011994.0  
2017-01-06   649970.0  58723.0  3011994.0  

[5 rows x 22 columns]

**N.B.** I consider the *Hadramaut* province together with *Socotra* province, then I sum the two populations. 

In [9]:
df.Yemen.Hadramaut = df.Yemen.Hadramaut + df.Yemen.Socotra
df.drop(columns = "Socotra", axis = 1, level = 1, inplace = True)
df.columns = pd.MultiIndex.from_tuples(list(map(lambda x: tuple(list(x) + ["Population"]), df.columns)))
df.columns.rename("Country", level = 0, inplace = True)
df.columns.rename("AdminStrata", level = 1, inplace = True)
df.columns.rename("Indicator", level = 2, inplace = True)
df = df.droplevel(level = 0, axis = 1)
df

AdminStrata,Abyan,Aden,Al Bayda,Al Dhale'e,Al Hudaydah,Al Jawf,Al Maharah,Al Mahwit,Amanat Al Asimah,Amran,...,Hadramaut,Hajjah,Ibb,Lahj,Marib,Raymah,Sa'ada,Sana'a,Shabwah,Taizz
Indicator,Population,Population,Population,Population,Population,Population,Population,Population,Population,Population,...,Population,Population,Population,Population,Population,Population,Population,Population,Population,Population
Datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-02,583193.0,953907.0,770257.0,753302.0,3315493.0,589218.0,159396.0,749952.0,2963191.0,1173354.0,...,1468072.0,2187109.0,3016766.0,1027981.0,368454.0,622101.0,923595.0,1497112.0,649970.0,3011994.0
2017-01-03,583193.0,953907.0,770257.0,753302.0,3315493.0,589218.0,159396.0,749952.0,2963191.0,1173354.0,...,1468072.0,2187109.0,3016766.0,1027981.0,368454.0,622101.0,923595.0,1497112.0,649970.0,3011994.0
2017-01-04,583193.0,953907.0,770257.0,753302.0,3315493.0,589218.0,159396.0,749952.0,2963191.0,1173354.0,...,1468072.0,2187109.0,3016766.0,1027981.0,368454.0,622101.0,923595.0,1497112.0,649970.0,3011994.0
2017-01-05,583193.0,953907.0,770257.0,753302.0,3315493.0,589218.0,159396.0,749952.0,2963191.0,1173354.0,...,1468072.0,2187109.0,3016766.0,1027981.0,368454.0,622101.0,923595.0,1497112.0,649970.0,3011994.0
2017-01-06,583193.0,953907.0,770257.0,753302.0,3315493.0,589218.0,159396.0,749952.0,2963191.0,1173354.0,...,1468072.0,2187109.0,3016766.0,1027981.0,368454.0,622101.0,923595.0,1497112.0,649970.0,3011994.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-28,583193.0,953907.0,770257.0,753302.0,3315493.0,589218.0,159396.0,749952.0,2963191.0,1173354.0,...,1468072.0,2187109.0,3016766.0,1027981.0,368454.0,622101.0,923595.0,1497112.0,649970.0,3011994.0
2020-10-29,583193.0,953907.0,770257.0,753302.0,3315493.0,589218.0,159396.0,749952.0,2963191.0,1173354.0,...,1468072.0,2187109.0,3016766.0,1027981.0,368454.0,622101.0,923595.0,1497112.0,649970.0,3011994.0


In [10]:
df.to_csv(path_to_save_data + "Yemen.csv", index_label = False)